In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import scipy.stats as st
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#ds=xr.open_dataset(dir+'extreme_days_ghcnd_fall_99_80_annual.nc')
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
years=[x.year for x in dates_unique]
print(years)
yrs_neusa=np.arange(1979,2020,1)

ds4=xr.open_dataset(dir+'neusa_ep_days_stats_var95const_station95.nc')
ar_yesno4=ds4['ar_yesno'].values.tolist()
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['ivt_tc_remnants_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['ar_tc_remnant_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()

print(tc_remnant_ar_combo_yesno4[145])
print(ar_yesno4[145])
print(dates_unique[145])

#these dates are non-TC-dominant EP days that have been associated with TC influences in the preceding 3 days
dates_remote_tc_inf=[dt.datetime(1979,9,21,6),dt.datetime(1987,9,9,6),dt.datetime(1989,9,19,6),
                     dt.datetime(1996,9,15,6),dt.datetime(2002,9,16,6),
                     dt.datetime(2002,9,28,6),dt.datetime(2003,9,2,6),dt.datetime(2003,9,4,6),
                     dt.datetime(2005,9,1,6),dt.datetime(2005,10,7,6),dt.datetime(2005,10,8,6),
                     dt.datetime(2005,10,9,6),dt.datetime(2005,10,15,6),dt.datetime(2009,11,12,6),
                     dt.datetime(2010,10,1,6),dt.datetime(2011,9,7,6),dt.datetime(2011,9,8,6),
                     dt.datetime(2015,9,30,6),dt.datetime(2015,10,1,6),dt.datetime(2017,10,29,6),
                     dt.datetime(2017,10,30,6),dt.datetime(2018,9,9,6),dt.datetime(2018,9,10,6),
                     dt.datetime(2018,9,19,6)]
remote_tc_inf_yesno4=[]
for i in range(len(dates_unique)):
    if dates_unique[i] in dates_remote_tc_inf:
        remote_tc_inf_yesno4.append(1)
    else:
        remote_tc_inf_yesno4.append(0)
print(sum(remote_tc_inf_yesno4))

In [ ]:
#can separate precip obs into northern and southern regions if desired

zipped=list(zip(dates,obs,lats,lons))
zipped_north=[x for x in zipped if x[2]>=42.]
zipped_south=[x for x in zipped if x[2]<42.]

#****
#zipped=zipped_north
#****

mean_obs_day_list=[]
max_obs_day_list=[]
for i in range(len(dates_unique)):
    date=dates_unique[i]
    select_day=[x for x in zipped if x[0]==date]
    obs_day=[x[1] for x in select_day]
    mean_obs_day=float(sum(obs_day))/float(len(select_day))
    mean_obs_day_list.append(mean_obs_day)
    max_obs_day_list.append(max(obs_day))

In [ ]:
#specify whether there are TC influences or not within non-TC categories
no_tc_inf_yesno4=[]
ar_notc_yesno4=[]
ivt_notc_yesno4=[]
other_notc_yesno4=[]
for i in range(len(remote_tc_inf_yesno4)):
    if tc_yesno4[i]==1 or tc_ar_combo_yesno4[i]==1 or tc_remnant_ar_combo_yesno4[i]==1 or tc_remnants_yesno4==1 or tc_linked_ar_yesno4[i]==1 or tc_remnant_linked_ar_yesno4[i]==1 or tc_linked_ivt_yesno4[i]==1 or tc_remnant_linked_ivt_yesno4[i]==1 or remote_tc_inf_yesno4[i]==1:
        no_tc_inf_yesno4.append(0)
        ar_notc_yesno4.append(0)
        ivt_notc_yesno4.append(0)
        other_notc_yesno4.append(0)
    else:
        no_tc_inf_yesno4.append(1)
        if ar_yesno4[i]==1:
            ar_notc_yesno4.append(1)
        else:
            ar_notc_yesno4.append(0)
        if ivt_yesno4[i]==1:
            ivt_notc_yesno4.append(1)
        else:
            ivt_notc_yesno4.append(0)
        if other_yesno4[i]==1:
            other_notc_yesno4.append(1)
        else:
            other_notc_yesno4.append(0)

In [ ]:
#Find dates and precip etc.

zipped_t1_years=list(zip(years,ar_yesno4,mean_obs_day_list))#ar-related = green
zipped_t2_years=list(zip(years,tc_linked_ar_yesno4,mean_obs_day_list))
zipped_t3_years=list(zip(years,tc_remnant_linked_ar_yesno4,mean_obs_day_list))
zipped_t4_years=list(zip(years,tc_yesno4,mean_obs_day_list))#tc-related = blue
zipped_t5_years=list(zip(years,tc_ar_combo_yesno4,mean_obs_day_list))
zipped_t6_years=list(zip(years,tc_remnant_ar_combo_yesno4,mean_obs_day_list))
zipped_t7_years=list(zip(years,tc_remnants_yesno4,mean_obs_day_list))
zipped_t8_years=list(zip(years,ivt_yesno4,mean_obs_day_list))#other IVT-related = grey
zipped_t9_years=list(zip(years,tc_linked_ivt_yesno4,mean_obs_day_list))
zipped_t10_years=list(zip(years,tc_remnant_linked_ivt_yesno4,mean_obs_day_list))
zipped_t11_years=list(zip(years,other_yesno4,mean_obs_day_list))#unspecified = brown

zipped_t12_years=list(zip(years,remote_tc_inf_yesno4,mean_obs_day_list))#unspecified = brown

zipped_t13_years=list(zip(years,no_tc_inf_yesno4,mean_obs_day_list))
zipped_t14_years=list(zip(years,ar_notc_yesno4,mean_obs_day_list))
zipped_t15_years=list(zip(years,ivt_notc_yesno4,mean_obs_day_list))
zipped_t16_years=list(zip(years,other_notc_yesno4,mean_obs_day_list))

freq_t1_list=[]
freq_t2_list=[]
freq_t3_list=[]
freq_t4_list=[]
freq_t5_list=[]
freq_t6_list=[]
freq_t7_list=[]
freq_t8_list=[]
freq_t9_list=[]
freq_t10_list=[]
freq_t11_list=[]
freq_t12_list=[]
freq_t13_list=[]
freq_t14_list=[]
freq_t15_list=[]
freq_t16_list=[]

precip_t1_list=[]
precip_t2_list=[]
precip_t3_list=[]
precip_t4_list=[]
precip_t5_list=[]
precip_t6_list=[]
precip_t7_list=[]
precip_t8_list=[]
precip_t9_list=[]
precip_t10_list=[]
precip_t11_list=[]
precip_t12_list=[]
precip_t13_list=[]
precip_t14_list=[]
precip_t15_list=[]
precip_t16_list=[]
for i in range(len(yrs_neusa)):
    year=1979+i
    select_t1=[x for x in zipped_t1_years if x[0]==year and x[1]==1]
    select_t2=[x for x in zipped_t2_years if x[0]==year and x[1]==1]
    select_t3=[x for x in zipped_t3_years if x[0]==year and x[1]==1]
    select_t4=[x for x in zipped_t4_years if x[0]==year and x[1]==1]
    select_t5=[x for x in zipped_t5_years if x[0]==year and x[1]==1]
    select_t6=[x for x in zipped_t6_years if x[0]==year and x[1]==1]
    select_t7=[x for x in zipped_t7_years if x[0]==year and x[1]==1]
    select_t8=[x for x in zipped_t8_years if x[0]==year and x[1]==1]
    select_t9=[x for x in zipped_t9_years if x[0]==year and x[1]==1]
    select_t10=[x for x in zipped_t10_years if x[0]==year and x[1]==1]
    select_t11=[x for x in zipped_t11_years if x[0]==year and x[1]==1]
    select_t12=[x for x in zipped_t12_years if x[0]==year and x[1]==1]
    select_t13=[x for x in zipped_t13_years if x[0]==year and x[1]==1]
    select_t14=[x for x in zipped_t14_years if x[0]==year and x[1]==1]
    select_t15=[x for x in zipped_t15_years if x[0]==year and x[1]==1]
    select_t16=[x for x in zipped_t16_years if x[0]==year and x[1]==1]
    
    freq_t1=len(select_t1)
    freq_t2=len(select_t2)
    freq_t3=len(select_t3)
    freq_t4=len(select_t4)
    freq_t5=len(select_t5)
    freq_t6=len(select_t6)
    freq_t7=len(select_t7)
    freq_t8=len(select_t8)
    freq_t9=len(select_t9)
    freq_t10=len(select_t10)
    freq_t11=len(select_t11)
    freq_t12=len(select_t12)
    freq_t13=len(select_t13)
    freq_t14=len(select_t14)
    freq_t15=len(select_t15)
    freq_t16=len(select_t16)
    freq_t1_list.append(freq_t1)
    freq_t2_list.append(freq_t2)
    freq_t3_list.append(freq_t3)
    freq_t4_list.append(freq_t4)
    freq_t5_list.append(freq_t5)
    freq_t6_list.append(freq_t6)
    freq_t7_list.append(freq_t7)
    freq_t8_list.append(freq_t8)
    freq_t9_list.append(freq_t9)
    freq_t10_list.append(freq_t10)
    freq_t11_list.append(freq_t11)
    freq_t12_list.append(freq_t12)
    freq_t13_list.append(freq_t13)
    freq_t14_list.append(freq_t14)
    freq_t15_list.append(freq_t15)
    freq_t16_list.append(freq_t16)
    
    precip_t1=[x[2] for x in select_t1]
    precip_t2=[x[2] for x in select_t2]
    precip_t3=[x[2] for x in select_t3]
    precip_t4=[x[2] for x in select_t4]
    precip_t5=[x[2] for x in select_t5]
    precip_t6=[x[2] for x in select_t6]
    precip_t7=[x[2] for x in select_t7]
    precip_t8=[x[2] for x in select_t8]
    precip_t9=[x[2] for x in select_t9]
    precip_t10=[x[2] for x in select_t10]
    precip_t11=[x[2] for x in select_t11]
    precip_t12=[x[2] for x in select_t12]
    precip_t13=[x[2] for x in select_t13]
    precip_t14=[x[2] for x in select_t14]
    precip_t15=[x[2] for x in select_t15]
    precip_t16=[x[2] for x in select_t16]
    
    precip_sum_t1=sum(precip_t1)
    precip_sum_t2=sum(precip_t2)
    precip_sum_t3=sum(precip_t3)
    precip_sum_t4=sum(precip_t4)
    precip_sum_t5=sum(precip_t5)
    precip_sum_t6=sum(precip_t6)
    precip_sum_t7=sum(precip_t7)
    precip_sum_t8=sum(precip_t8)
    precip_sum_t9=sum(precip_t9)
    precip_sum_t10=sum(precip_t10)
    precip_sum_t11=sum(precip_t11)
    precip_sum_t12=sum(precip_t12)
    precip_sum_t13=sum(precip_t13)
    precip_sum_t14=sum(precip_t14)
    precip_sum_t15=sum(precip_t15)
    precip_sum_t16=sum(precip_t16)
    
    precip_t1_list.append(precip_sum_t1)
    precip_t2_list.append(precip_sum_t2)
    precip_t3_list.append(precip_sum_t3)
    precip_t4_list.append(precip_sum_t4)
    precip_t5_list.append(precip_sum_t5)
    precip_t6_list.append(precip_sum_t6)
    precip_t7_list.append(precip_sum_t7)
    precip_t8_list.append(precip_sum_t8)
    precip_t9_list.append(precip_sum_t9)
    precip_t10_list.append(precip_sum_t10)
    precip_t11_list.append(precip_sum_t11)
    precip_t12_list.append(precip_sum_t12)
    precip_t13_list.append(precip_sum_t13)
    precip_t14_list.append(precip_sum_t14)
    precip_t15_list.append(precip_sum_t15)
    precip_t16_list.append(precip_sum_t16)

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm

from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
linreg0=st.linregress(yrs_neusa,freq_t2_list)
slope0=linreg0[0]
intercept0=linreg0[1]
p_value0=linreg0[3]
mk0=mk_test(freq_t2_list,alpha=0.05)
if mk0[0] in ['increasing','decreasing']:
    mk0_str='MK-test = significant'
else:
    mk0_str='MK-test = insignificant'
if intercept0<0:
    eq_str0='y='+str(round(slope0,2))+'x-'+str(abs(round(intercept0,2)))+', p='+str(round(p_value0,4))+' ) '
else:
    eq_str0='y='+str(round(slope0,2))+'x+'+str(abs(round(intercept0,2)))+', p='+str(round(p_value0,4))+' ) '


linreg=st.linregress(yrs_neusa,freq_t1_list)
slope1=linreg[0]
intercept1=linreg[1]
p_value1=linreg[3]
mk1=mk_test(freq_t1_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '

ar_total_list=[x+y+z for x,y,z in zip(freq_t1_list,freq_t2_list,freq_t3_list)]
linreg2=st.linregress(yrs_neusa,ar_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(ar_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
    
fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,ar_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+' | '+mk2_str,zorder=10)
ax.bar(yrs_neusa,freq_t1_list,color='g',label='Pure AR ('+eq_str+' | '+mk1_str)
ax.bar(yrs_neusa,freq_t2_list,color='limegreen',bottom=freq_t1_list,label='TC-linked AR ('+eq_str0+' | '+mk0_str)
ax.bar(yrs_neusa,freq_t3_list,color='palegreen',bottom=[x+y for x,y in zip(freq_t1_list,freq_t2_list)],label='TC remnant-linked AR')
ax.set_title('AR-dominant',fontsize=26)#,pad=10)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Number of Days',fontsize=18,labelpad=15)
ax.set_ylim(0,9)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=15)
plt.yticks([1,2,3,4,5,6,7,8,9])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew1.png')

In [ ]:
linreg0=st.linregress(yrs_neusa,precip_t2_list)
slope0=linreg0[0]
intercept0=linreg0[1]
p_value0=linreg0[3]
mk0=mk_test(precip_t2_list,alpha=0.05)
if mk0[0] in ['increasing','decreasing']:
    mk0_str='MK-test = significant'
else:
    mk0_str='MK-test = insignificant'
if intercept0<0:
    eq_str0='y='+str(round(slope0,2))+'x-'+str(abs(round(intercept0,2)))+', p='+str(round(p_value0,4))+' ) '
else:
    eq_str0='y='+str(round(slope0,2))+'x+'+str(abs(round(intercept0,2)))+', p='+str(round(p_value0,4))+' ) '


linreg=st.linregress(yrs_neusa,precip_t1_list)
slope1=linreg[0]
intercept1=linreg[1]
p_value1=linreg[3]
mk1=mk_test(precip_t1_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '

ar_total_list=[x+y+z for x,y,z in zip(precip_t1_list,precip_t2_list,precip_t3_list)]
linreg2=st.linregress(yrs_neusa,ar_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(ar_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
    
fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,ar_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+' | '+mk2_str,zorder=10)
ax.bar(yrs_neusa,precip_t1_list,color='g',label='Pure AR ('+eq_str+' | '+mk1_str)
ax.bar(yrs_neusa,precip_t2_list,color='limegreen',bottom=precip_t1_list,label='TC-linked AR ('+eq_str0+' | '+mk0_str)
ax.bar(yrs_neusa,precip_t3_list,color='palegreen',bottom=[x+y for x,y in zip(precip_t1_list,precip_t2_list)],label='TC remnant-linked AR')
ax.set_title('AR-Dominant Weather Types',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Mean precipitation (mm)',fontsize=18,labelpad=15)
ax.set_ylim(0,200)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=15)
plt.yticks([25,50,75,100,125,150,175,200])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew2.png')

In [ ]:
#find fraction of total precip and trend from this category

all_total_list=[x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11 for x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11 in zip(precip_t1_list,precip_t2_list,precip_t3_list,
                                                                                                 precip_t4_list,precip_t5_list,precip_t6_list,
                                                                                                 precip_t7_list,precip_t8_list,precip_t9_list,
                                                                                                 precip_t10_list,precip_t11_list)]

print(all_total_list)
all_total_mean=float(sum(all_total_list))/float(len(all_total_list))
ar_mean=float(sum(ar_total_list))/float(len(ar_total_list))

linreg3=st.linregress(yrs_neusa,all_total_list)
slope3=linreg3[0]

fraction_ar_precip=ar_mean/all_total_mean
fraction_ar_trend=(slope2*ar_mean)/(slope3*all_total_mean)
print(fraction_ar_precip)
print(fraction_ar_trend)
print(slope2)

In [ ]:
linreg1=st.linregress(yrs_neusa,freq_t4_list)
slope1=linreg1[0]
intercept1=linreg1[1]
p_value1=linreg1[3]
mk1=mk_test(freq_t4_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str1='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str1='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '

tc_total_list=[x+y+z+w for x,y,z,w in zip(freq_t4_list,freq_t5_list,freq_t6_list,freq_t7_list)]
linreg2=st.linregress(yrs_neusa,tc_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(tc_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '

    
linreg3=st.linregress(yrs_neusa,freq_t7_list)
slope3=linreg3[0]
intercept3=linreg3[1]
p_value3=linreg3[3]
mk3=mk_test(freq_t7_list,alpha=0.05)
if mk3[0] in ['increasing','decreasing']:
    mk3_str='MK-test = significant'
else:
    mk3_str='MK-test = insignificant'
if intercept3<0:
    eq_str3='y='+str(round(slope3,2))+'x-'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '
else:
    eq_str3='y='+str(round(slope3,2))+'x+'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '
    
linreg4=st.linregress(yrs_neusa,freq_t6_list)
slope4=linreg4[0]
intercept4=linreg4[1]
p_value4=linreg4[3]
mk4=mk_test(freq_t6_list,alpha=0.05)
if mk4[0] in ['increasing','decreasing']:
    mk4_str='MK-test = significant'
else:
    mk4_str='MK-test = insignificant'
if intercept4<0:
    eq_str4='y='+str(round(slope4,2))+'x-'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '
else:
    eq_str4='y='+str(round(slope4,2))+'x+'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '
    
linreg5=st.linregress(yrs_neusa,freq_t5_list)
slope5=linreg5[0]
intercept5=linreg5[1]
p_value5=linreg5[3]
mk5=mk_test(freq_t5_list,alpha=0.05)
if mk5[0] in ['increasing','decreasing']:
    mk5_str='MK-test = significant'
else:
    mk5_str='MK-test = insignificant'
if intercept5<0:
    eq_str5='y='+str(round(slope5,2))+'x-'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '
else:
    eq_str5='y='+str(round(slope5,2))+'x+'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '
    
fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,tc_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+' | '+mk2_str,zorder=10)
ax.bar(yrs_neusa,freq_t4_list,color='darkred',label='Pure TC ('+eq_str1+' | '+mk1_str)
ax.bar(yrs_neusa,freq_t7_list,color='r',bottom=freq_t4_list,label='TC remnants ('+eq_str3+' | '+mk3_str)
ax.bar(yrs_neusa,freq_t6_list,color='salmon',bottom=[x+y for x,y in zip(freq_t4_list,freq_t7_list)],label='TC remnant - AR combination ('+eq_str4+' | '+mk4_str)
ax.bar(yrs_neusa,freq_t5_list,color='mistyrose',bottom=[x+y+z for x,y,z in zip(freq_t4_list,freq_t7_list,freq_t6_list)],label='TC-AR combination ('+eq_str5+' | '+mk5_str)


ax.set_title('TC-Dominant Weather Types',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Number of Days',fontsize=18,labelpad=15)
ax.set_ylim(0,9)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=15)
plt.yticks([1,2,3,4,5,6,7,8,9])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew3.png')

In [ ]:
linreg1=st.linregress(yrs_neusa,precip_t4_list)
slope1=linreg1[0]
intercept1=linreg1[1]
p_value1=linreg1[3]
mk1=mk_test(precip_t4_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str1='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str1='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '

tc_total_list=[x+y+z+w for x,y,z,w in zip(precip_t4_list,precip_t5_list,precip_t6_list,precip_t7_list)]
linreg2=st.linregress(yrs_neusa,tc_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(tc_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '

linreg3=st.linregress(yrs_neusa,precip_t7_list)
slope3=linreg3[0]
intercept3=linreg3[1]
p_value3=linreg3[3]
mk3=mk_test(precip_t7_list,alpha=0.05)
if mk3[0] in ['increasing','decreasing']:
    mk3_str='MK-test = significant'
else:
    mk3_str='MK-test = insignificant'
if intercept3<0:
    eq_str3='y='+str(round(slope3,2))+'x-'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '
else:
    eq_str3='y='+str(round(slope3,2))+'x+'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '

linreg4=st.linregress(yrs_neusa,precip_t6_list)
slope4=linreg4[0]
intercept4=linreg4[1]
p_value4=linreg4[3]
mk4=mk_test(precip_t6_list,alpha=0.05)
if mk4[0] in ['increasing','decreasing']:
    mk4_str='MK-test = significant'
else:
    mk4_str='MK-test = insignificant'
if intercept4<0:
    eq_str4='y='+str(round(slope4,2))+'x-'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '
else:
    eq_str4='y='+str(round(slope4,2))+'x+'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '
    
linreg5=st.linregress(yrs_neusa,precip_t5_list)
slope5=linreg5[0]
intercept5=linreg5[1]
p_value5=linreg5[3]
mk5=mk_test(precip_t5_list,alpha=0.05)
if mk5[0] in ['increasing','decreasing']:
    mk5_str='MK-test = significant'
else:
    mk5_str='MK-test = insignificant'
if intercept5<0:
    eq_str5='y='+str(round(slope5,2))+'x-'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '
else:
    eq_str5='y='+str(round(slope5,2))+'x+'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '
    
    
fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,tc_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+' | '+mk2_str,zorder=5)
ax.bar(yrs_neusa,precip_t4_list,color='darkred',label='Pure TC ('+eq_str1+' | '+mk1_str)
ax.bar(yrs_neusa,precip_t7_list,color='r',bottom=precip_t4_list,label='TC remnants ('+eq_str3+' | '+mk3_str)
ax.bar(yrs_neusa,precip_t6_list,color='salmon',bottom=[x+y for x,y in zip(precip_t4_list,precip_t7_list)],label='TC remnant - AR combination ('+eq_str4+' | '+mk4_str)
ax.bar(yrs_neusa,precip_t5_list,color='mistyrose',bottom=[x+y+z for x,y,z in zip(precip_t4_list,precip_t7_list,precip_t6_list)],label='TC-AR combination ('+eq_str5+' | '+mk5_str)



ax.set_title('TC-Dominant Weather Types',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Mean precipitation (mm)',fontsize=18,labelpad=15)
ax.set_ylim(0,200)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=15,framealpha=0.8)
plt.yticks([25,50,75,100,125,150,175,200])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew4.png')
#fig.savefig(dir+'neusa_wtypes_95bars4_split2.png')

In [ ]:
#find fraction of total precip and trend from this category

all_total_list=[x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11 for x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11 in zip(precip_t1_list,precip_t2_list,precip_t3_list,
                                                                                                 precip_t4_list,precip_t5_list,precip_t6_list,
                                                                                                 precip_t7_list,precip_t8_list,precip_t9_list,
                                                                                                 precip_t10_list,precip_t11_list)]

print(all_total_list)
all_total_mean=float(sum(all_total_list))/float(len(all_total_list))
tc_mean=float(sum(tc_total_list))/float(len(tc_total_list))

linreg3=st.linregress(yrs_neusa,all_total_list)
slope3=linreg3[0]

fraction_tc_precip=tc_mean/all_total_mean
fraction_tc_trend=(slope2*tc_mean)/(slope3*all_total_mean)
print(fraction_tc_precip)
print(fraction_tc_trend)

print(slope2)

In [ ]:
linreg1=st.linregress(yrs_neusa,freq_t8_list)
slope1=linreg1[0]
intercept1=linreg1[1]
p_value1=linreg1[3]
mk1=mk_test(freq_t8_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str1='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str1='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
    
linreg2=st.linregress(yrs_neusa,freq_t9_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(freq_t9_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept1<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
    
linreg3=st.linregress(yrs_neusa,freq_t11_list)
slope3=linreg3[0]
intercept3=linreg3[1]
p_value3=linreg3[3]
mk3=mk_test(freq_t11_list,alpha=0.05)
if mk3[0] in ['increasing','decreasing']:
    mk3_str='MK-test = significant'
else:
    mk3_str='MK-test = insignificant'
if intercept1<0:
    eq_str3='y='+str(round(slope3,2))+'x-'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '
else:
    eq_str3='y='+str(round(slope3,2))+'x+'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '

other_total_list=[x+y+z+w for x,y,z,w in zip(freq_t8_list,freq_t9_list,freq_t10_list,freq_t11_list)]
linreg4=st.linregress(yrs_neusa,other_total_list)
slope4=linreg4[0]
intercept4=linreg4[1]
p_value4=linreg4[3]
mk4=mk_test(other_total_list,alpha=0.05)
if mk4[0] in ['increasing','decreasing']:
    mk4_str='MK-test = significant'
else:
    mk4_str='MK-test = insignificant'
if intercept2<0:
    eq_str4='y='+str(round(slope4,2))+'x-'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '
else:
    eq_str4='y='+str(round(slope4,2))+'x+'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '

    
linreg5=st.linregress(yrs_neusa,freq_t10_list)
slope5=linreg5[0]
intercept5=linreg5[1]
p_value5=linreg5[3]
mk5=mk_test(freq_t10_list,alpha=0.05)
if mk5[0] in ['increasing','decreasing']:
    mk5_str='MK-test = significant'
else:
    mk5_str='MK-test = insignificant'
if intercept1<0:
    eq_str5='y='+str(round(slope5,2))+'x-'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '
else:
    eq_str5='y='+str(round(slope5,2))+'x+'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '


fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)
print(freq_t8_list)
print(freq_t9_list)
print(freq_t10_list)
print(freq_t11_list)
ax.bar(yrs_neusa,other_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str4+' | '+mk4_str,zorder=10)
ax.bar(yrs_neusa,freq_t8_list,color='darkblue',label='Pure extreme IVT ('+eq_str1+' | '+mk1_str)
ax.bar(yrs_neusa,freq_t9_list,color='b',bottom=freq_t8_list,label='TC-linked extreme IVT ('+eq_str2+' | '+mk2_str)
#ax.bar(yrs_neusa,freq_t10_list,color='dodgerblue',bottom=[x+y for x,y in zip(freq_t8_list,freq_t9_list)],label='TC remnant-linked extreme IVT ('+eq_str5+' | '+mk5_str)
ax.bar(yrs_neusa,freq_t11_list,color='cyan',bottom=[x+y+z for x,y,z in zip(freq_t8_list,freq_t9_list,freq_t10_list)],label='Unspecified ('+eq_str3+' | '+mk3_str)




ax.set_title('Other Weather Types',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Number of Days',fontsize=18,labelpad=15)
ax.set_ylim(0,9)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=15)
plt.yticks([1,2,3,4,5,6,7,8,9])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew5.png')

In [ ]:
linreg1=st.linregress(yrs_neusa,precip_t8_list)
slope1=linreg1[0]
intercept1=linreg1[1]
p_value1=linreg1[3]
mk1=mk_test(precip_t8_list,alpha=0.05)
if mk1[0] in ['increasing','decreasing']:
    mk1_str='MK-test = significant'
else:
    mk1_str='MK-test = insignificant'
if intercept1<0:
    eq_str1='y='+str(round(slope1,2))+'x-'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
else:
    eq_str1='y='+str(round(slope1,2))+'x+'+str(abs(round(intercept1,2)))+', p='+str(round(p_value1,4))+' ) '
    
linreg2=st.linregress(yrs_neusa,precip_t9_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(precip_t9_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept1<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+' ) '
    
linreg3=st.linregress(yrs_neusa,precip_t11_list)
slope3=linreg3[0]
intercept3=linreg3[1]
p_value3=linreg3[3]
mk3=mk_test(precip_t11_list,alpha=0.05)
if mk3[0] in ['increasing','decreasing']:
    mk3_str='MK-test = significant'
else:
    mk3_str='MK-test = insignificant'
if intercept1<0:
    eq_str3='y='+str(round(slope3,2))+'x-'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '
else:
    eq_str3='y='+str(round(slope3,2))+'x+'+str(abs(round(intercept3,2)))+', p='+str(round(p_value3,4))+' ) '

other_total_list=[x+y+z+w for x,y,z,w in zip(precip_t8_list,precip_t9_list,precip_t10_list,precip_t11_list)]
linreg4=st.linregress(yrs_neusa,other_total_list)
slope4=linreg4[0]
intercept4=linreg4[1]
p_value4=linreg4[3]
mk4=mk_test(other_total_list,alpha=0.05)
if mk4[0] in ['increasing','decreasing']:
    mk4_str='MK-test = significant'
else:
    mk4_str='MK-test = insignificant'
if intercept2<0:
    eq_str4='y='+str(round(slope4,2))+'x-'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '
else:
    eq_str4='y='+str(round(slope4,2))+'x+'+str(abs(round(intercept4,2)))+', p='+str(round(p_value4,4))+' ) '

linreg5=st.linregress(yrs_neusa,precip_t10_list)
slope5=linreg5[0]
intercept5=linreg5[1]
p_value5=linreg5[3]
mk5=mk_test(precip_t10_list,alpha=0.05)
if mk5[0] in ['increasing','decreasing']:
    mk5_str='MK-test = significant'
else:
    mk5_str='MK-test = insignificant'
if intercept1<0:
    eq_str5='y='+str(round(slope5,2))+'x-'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '
else:
    eq_str5='y='+str(round(slope5,2))+'x+'+str(abs(round(intercept5,2)))+', p='+str(round(p_value5,4))+' ) '


    
fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)

ax.bar(yrs_neusa,other_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str4+' | '+mk4_str,zorder=10)
ax.bar(yrs_neusa,precip_t8_list,color='darkblue',label='Pure extreme IVT ('+eq_str1+' | '+mk1_str)
ax.bar(yrs_neusa,precip_t9_list,color='b',bottom=precip_t8_list,label='TC-linked extreme IVT ('+eq_str2+' | '+mk2_str)
#ax.bar(yrs_neusa,precip_t10_list,color='dodgerblue',bottom=[x+y for x,y in zip(freq_t8_list,freq_t9_list)],label='TC remnant-linked extreme IVT ('+eq_str5+' | '+mk5_str)
ax.bar(yrs_neusa,precip_t11_list,color='cyan',bottom=[x+y+z for x,y,z in zip(precip_t8_list,precip_t9_list,precip_t10_list)],label='Unspecified ('+eq_str3+' | '+mk3_str)




ax.set_title('Other Weather Types',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Mean precipitation (mm)',fontsize=18,labelpad=15)
ax.set_ylim(0,200)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=15)
plt.yticks([25,50,75,100,125,150,175,200])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew6.png')
#fig.savefig(dir+'neusa_wtypes_95bars6_split2.png')

In [ ]:
#find fraction of total precip and trend from this category

all_total_list=[x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11 for x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11 in zip(precip_t1_list,precip_t2_list,precip_t3_list,
                                                                                                 precip_t4_list,precip_t5_list,precip_t6_list,
                                                                                                 precip_t7_list,precip_t8_list,precip_t9_list,
                                                                                                 precip_t10_list,precip_t11_list)]

print(all_total_list)
all_total_mean=float(sum(all_total_list))/float(len(all_total_list))
other_mean=float(sum(other_total_list))/float(len(other_total_list))

linreg3=st.linregress(yrs_neusa,all_total_list)
slope3=linreg3[0]

fraction_other_precip=other_mean/all_total_mean
fraction_other_trend=(slope4*other_mean)/(slope3*all_total_mean)
print(fraction_other_precip)
print(fraction_other_trend)

print(slope4)

print(slope3)

In [ ]:
tc=0.5243
ar=0.444
other = 0.288
all1=1.256
print(tc+ar+other)
print(tc/all1)
print(ar/all1)
print(other/all1)

In [ ]:
#now factor in prior TC influences

tc_inf_total_list=[x1+x2+x3+x4+x5+x6+x7+x8+x9 for x1,x2,x3,x4,x5,x6,x7,x8,x9 in zip(freq_t4_list,freq_t5_list,freq_t6_list,freq_t7_list,freq_t2_list,freq_t3_list,freq_t9_list,freq_t10_list,freq_t12_list)]
linreg2=st.linregress(yrs_neusa,tc_inf_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(tc_inf_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'

fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,tc_inf_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+'\n'+mk2_str,zorder=10)
ax.bar(yrs_neusa,freq_t4_list,color='darkred',label='Pure TC')
ax.bar(yrs_neusa,freq_t7_list,color='r',bottom=freq_t4_list,label='TC remnants')
ax.bar(yrs_neusa,freq_t6_list,color='salmon',bottom=[x+y for x,y in zip(freq_t4_list,freq_t7_list)],label='TC remnant - AR combination')
ax.bar(yrs_neusa,freq_t5_list,color='mistyrose',bottom=[x+y+z for x,y,z in zip(freq_t4_list,freq_t7_list,freq_t6_list)],label='TC-AR combination')


ax.bar(yrs_neusa,freq_t2_list,color='limegreen',bottom=[x+y+z+w for x,y,z,w in zip(freq_t4_list,freq_t7_list,freq_t6_list,freq_t5_list)],label='TC-linked AR')
ax.bar(yrs_neusa,freq_t3_list,color='palegreen',bottom=[x+y+z+w+q for x,y,z,w,q in zip(freq_t4_list,freq_t7_list,freq_t6_list,freq_t5_list,freq_t2_list)],label='TC remnant-linked AR')

ax.bar(yrs_neusa,freq_t9_list,color='b',bottom=[x1+x2+x3+x4+x5+x6 for x1,x2,x3,x4,x5,x6 in zip(freq_t4_list,freq_t7_list,freq_t6_list,freq_t5_list,freq_t2_list,freq_t3_list)],label='TC-linked extreme IVT')
#ax.bar(yrs_neusa,freq_t10_list,color='b',bottom=[x1+x2+x3+x4+x5+x6+x7 for x1,x2,x3,x4,x5,x6,x7 in zip(freq_t4_list,freq_t7_list,freq_t6_list,freq_t5_list,freq_t2_list,freq_t3_list,freq_t9_list)],label='TC remnant-linked extreme IVT')
ax.bar(yrs_neusa,freq_t12_list,color='tan',bottom=[x1+x2+x3+x4+x5+x6+x7+x8 for x1,x2,x3,x4,x5,x6,x7,x8 in zip(freq_t4_list,freq_t7_list,freq_t6_list,freq_t5_list,freq_t2_list,freq_t3_list,freq_t9_list,freq_t10_list)],label='Prior TC influences')
print(freq_t10_list)

ax.set_title('Any TC influences: Frequency',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Number of Days',fontsize=18,labelpad=15)
ax.set_ylim(0,12)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=14)
plt.yticks([1,2,3,4,5,6,7,8,9,10,11])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew10.png')

In [ ]:
tc_inf_total_list=[x1+x2+x3+x4+x5+x6+x7+x8+x9 for x1,x2,x3,x4,x5,x6,x7,x8,x9 in zip(precip_t4_list,precip_t5_list,precip_t6_list,precip_t7_list,precip_t2_list,precip_t3_list,precip_t9_list,precip_t10_list,precip_t12_list)]
linreg2=st.linregress(yrs_neusa,tc_inf_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(tc_inf_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'

fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.bar(yrs_neusa,tc_inf_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+'\n'+mk2_str,zorder=10)
ax.bar(yrs_neusa,precip_t4_list,color='darkred',label='Pure TC')
ax.bar(yrs_neusa,precip_t7_list,color='r',bottom=freq_t4_list,label='TC remnants')
ax.bar(yrs_neusa,precip_t6_list,color='salmon',bottom=[x+y for x,y in zip(precip_t4_list,precip_t7_list)],label='TC remnant - AR combination')
ax.bar(yrs_neusa,precip_t5_list,color='mistyrose',bottom=[x+y+z for x,y,z in zip(precip_t4_list,precip_t7_list,precip_t6_list)],label='TC-AR combination')


ax.bar(yrs_neusa,precip_t2_list,color='limegreen',bottom=[x+y+z+w for x,y,z,w in zip(precip_t4_list,precip_t7_list,precip_t6_list,precip_t5_list)],label='TC-linked AR')
ax.bar(yrs_neusa,precip_t3_list,color='palegreen',bottom=[x+y+z+w+q for x,y,z,w,q in zip(precip_t4_list,precip_t7_list,precip_t6_list,precip_t5_list,precip_t2_list)],label='TC remnant-linked AR')

ax.bar(yrs_neusa,precip_t9_list,color='b',bottom=[x1+x2+x3+x4+x5+x6 for x1,x2,x3,x4,x5,x6 in zip(precip_t4_list,precip_t7_list,precip_t6_list,precip_t5_list,precip_t2_list,precip_t3_list)],label='TC-linked extreme IVT')
ax.bar(yrs_neusa,precip_t12_list,color='tan',bottom=[x1+x2+x3+x4+x5+x6+x7+x8 for x1,x2,x3,x4,x5,x6,x7,x8 in zip(precip_t4_list,precip_t7_list,precip_t6_list,precip_t5_list,precip_t2_list,precip_t3_list,precip_t9_list,precip_t10_list)],label='Prior TC influences')


ax.set_title('Any TC influences: Precipitation',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Mean precipitation (mm)',fontsize=18,labelpad=15)
ax.set_ylim(0,200)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=14)
plt.yticks([25,50,75,100,125,150,175,200])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew11.png')

In [ ]:
#no TC influences

tc_inf_total_list=[x1+x2+x3 for x1,x2,x3 in zip(freq_t14_list,freq_t15_list,freq_t16_list)]
linreg2=st.linregress(yrs_neusa,tc_inf_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(tc_inf_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'

fig=plt.figure(figsize=(14,10))
ax=plt.subplot(1,1,1)

ax.bar(yrs_neusa,tc_inf_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+'\n'+mk2_str,zorder=10)
ax.bar(yrs_neusa,freq_t14_list,color='g',label='Pure AR')
ax.bar(yrs_neusa,freq_t15_list,color='b',bottom=freq_t14_list,label='Pure extreme IVT')
ax.bar(yrs_neusa,freq_t16_list,color='cyan',bottom=[x+y for x,y in zip(freq_t14_list,freq_t15_list)],label='Unspecified')


ax.set_title('No TC influences: Frequency',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Number of Days',fontsize=18,labelpad=15)
ax.set_ylim(0,12)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=14)
plt.yticks([1,2,3,4,5,6,7,8,9,10,11])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew12.png')

In [ ]:
tc_inf_total_list=[x1+x2+x3 for x1,x2,x3 in zip(precip_t14_list,precip_t15_list,precip_t16_list)]
linreg2=st.linregress(yrs_neusa,tc_inf_total_list)
slope2=linreg2[0]
intercept2=linreg2[1]
p_value2=linreg2[3]
mk2=mk_test(tc_inf_total_list,alpha=0.05)
if mk2[0] in ['increasing','decreasing']:
    mk2_str='MK-test = significant'
else:
    mk2_str='MK-test = insignificant'
if intercept2<0:
    eq_str2='y='+str(round(slope2,2))+'x-'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'
else:
    eq_str2='y='+str(round(slope2,2))+'x+'+str(abs(round(intercept2,2)))+', p='+str(round(p_value2,4))+')'

fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)

ax.bar(yrs_neusa,tc_inf_total_list,color='None',edgecolor='k',linewidth=2.0,label='Total ('+eq_str2+'\n'+mk2_str,zorder=10)
ax.bar(yrs_neusa,precip_t14_list,color='g',label='Pure AR')
ax.bar(yrs_neusa,precip_t15_list,color='b',bottom=precip_t14_list,label='Pure extreme IVT')
ax.bar(yrs_neusa,precip_t16_list,color='cyan',bottom=[x+y for x,y in zip(precip_t14_list,precip_t15_list)],label='Unspecified')


ax.set_title('No TC influences: Precipitation',fontsize=26)
ax.tick_params(labelsize=16)
ax.set_xlabel('Year',fontsize=18,labelpad=15)
ax.set_ylabel('Mean precipitation (mm)',fontsize=18,labelpad=15)
ax.set_ylim(0,200)
ax.set_xlim(1978.5,2019.5)
plt.legend(loc='upper left',fontsize=14)
plt.yticks([25,50,75,100,125,150,175,200])

plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtypes_95barsnew13.png')